In [28]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from cvxpy import * 

In [29]:
def train_test_split(data, start_year):
    '''
    Split the data up into training and test periods
    
    Args: 
        data: a n stocks by t periods data-frame with a date-time index
        period: an integer representing which period that should be used
        
    Returns:
        tuple of training and testing data as pandas dataframes
        
    Usage: 
        training1, testing1 = train_test_split(stocks, 1)
    '''
    
    training = data[str(start_year):str(start_year + 4)]
    testing = data[str(start_year + 5)]
    return(training, testing)

In [30]:
def read_stocks():
    '''
    Get stock data for performing optimization. 
    Assumes data is in a t-by-n format with t months of observations
    along the rows and n stocks to choose from in columns.
    Assumes no missing values.
    
    Returns:
         Monthly returns data in a t-by-n formated dataframe with 
         a time-formatted index
    '''
    
    df = pd.read_csv('../../data/monthly_return.csv', names = ['s_' + str(x+1) for x in range(556)])
    df['date'] = pd.date_range('1/1/1986', periods=360, freq='M')
    df.set_index('date', inplace=True)
    
    return(df)

In [31]:
def read_factors3():
    '''
    Get fama and french factor data for performing optimization. 
    Assumes data is in a t-by-3 format with t months of observations
    along the rows and 3 factors to choose from in columns.
    Assumes no missing values.

    Returns:
         Monthly returns data in a t-by-n formated dataframe with 
         a time-formatted index
    '''
    
    df = pd.read_csv('../../data/F-F_Research_Data_3_Factors.csv'
                     , skiprows=3, parse_dates=True, nrows=1088)
    keep_dates = (df.loc[:,'Unnamed: 0'] >= 198601) & (df.loc[:,'Unnamed: 0'] <= 201512)
    df = df.loc[keep_dates, ['Mkt-RF', 'SMB', 'HML']] # Exclude risk free rate
    df['date'] = pd.date_range('1/1/1986', periods=360, freq='M')
    df.set_index('date', inplace=True)
    
    return(df)

In [32]:
def read_factors5():
    '''
    Get fama and french factor data for performing optimization. 
    Returns data is in a t-by-5 format with t months of observations
    along the rows and 5 factors to choose from in columns.
    Assumes no missing values.

    Returns:
         Monthly returns data in a t-by-5 formated dataframe with 
         a time-formatted index
    '''
    
    df = pd.read_csv('../../data/F-F_Research_Data_5_Factors.csv'
                     , skiprows=3, parse_dates=True, nrows=645)
    keep_dates = (df.loc[:,'Unnamed: 0'] >= 198601) & (df.loc[:,'Unnamed: 0'] <= 201512)
    df = df.loc[keep_dates, ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']] # Exclude risk free rate   
    df['date'] = pd.date_range('1/1/1986', periods=360, freq='M')
    df.set_index('date', inplace=True)
    
    return(df)

In [33]:
def fama_and_french(stocks, factors):
    '''
    Calculate factor loadings, factor covariance, and idiosyncratic risk 
    for a single training period
    
    Args:
        stocks: stocks for a single period as a t by n numpy array (this is training period)
        factors: factors for a single period as a t by k numpy array (this is training period)
    
    Returns:
        dictionary including factor loadings (n by k), factor covariance (k by k) and idiosyncratic risk (n by n) 
    '''
    
    # Number of stocks
    n = stocks.shape[1]
    
    # Number of factors
    k = factors.shape[1]
    
    # Factor loadings
    F = np.zeros(shape = (n, k + 1))

    # Idiosyncratic risk
    D = np.diag(np.zeros(shape = n))

    # Define the input for the regression
    X = factors
    X = sm.add_constant(X)

    # Loop through the stocks and calculate coefficients
    for i in range(n):

        # Select new stock each time
        y_i = stocks.iloc[:,i] 
        model_i = sm.OLS(y_i, X).fit() 

        # Including the alpha term and all k betas
        F[i,:] = model_i.params.values

        # Denominator is: (60 months) - (k factors) + (1 constant)
        D[i,i] = np.sum((y_i - model_i.predict(X))**2) / (60 - k + 1) 

    # The Factor Covariance Matrix
    Sigma_tilde = np.cov(F, rowvar=False)
    
    return(F, Sigma_tilde, D)

In [34]:
def get_equal_weights(n):
    '''
    Get a vector of equal weights
    
    Args:
        n: How many stocks are in your portfolio
        
    Returns:
        a n-by-1 matrix of 1/n weights
    '''
    
    x = np.asmatrix(np.ones(shape = (n, 1))) / n
    return(x)

In [35]:
def ret_cov_est(training):
    '''
    Estimate the returns and covariance for the training period.
    
    Args:
        training: a n stocks by t periods data-frame with a date-time index
    
    Returns:
        A tuple with a n-by-1 matrix of estimated returns 
        and a n-by-n matrix of estimated covariance. n is the number of stocks.
    '''
    r_hat = np.asmatrix(np.mean(training)).T
    Sigma = np.asmatrix(np.cov(training, rowvar=False))
    return(r_hat, Sigma)

In [36]:
def min_variance(r_hat, Sigma, tau, mu = None):
    '''
    For a given estimated return floor, get the weights that 
    minimize the variance with an l1 norm of the weights.
    
    Args:
        r_hat: estimated returns as a n-by-1 matrix
        Sigma: estimated covariance as a n-by-n matrix
        tau: tuning parameter. (Larger values promote more sparsity.)
        mu: the minimum return that the portfolio must beat
        
    Returns:
        optimal weights as a n-by-1 matrix
    '''
    
    # Define the variables
    n = r_hat.shape[0]
    x = Variable(n)
    ret = r_hat.T*x 
    risk = quad_form(x, Sigma)
    
    # Define the problem: Minimize variance for given returns threshold
    objective = Minimize(risk + tau*norm(x, 1))
    constraints = [sum_entries(x) == 1, x >= 0, ret >= mu]
    
    # Solve the problem
    prob = Problem(objective, constraints)
    prob.solve()
    
    # Get the values of interest
    minimal_risk = risk.value
    optimal_x = x.value
    
    # Handling rounding of x's
    optimal_x = np.around(optimal_x, decimals = 4)
    optimal_x =  np.asmatrix(optimal_x / sum(optimal_x))
    
    return(optimal_x)

In [37]:
def min_variance_factor(r_hat, F, Sigma_tilde, D, mu):
    
    # Define the variables
    n = F.shape[0]
    x = Variable(n)   # The weights
    f = F.T*x         # The factor loadings
    ret = r_hat.T*x 
    risk = quad_form(f, Sigma_tilde) + quad_form(x, D)
    
    # Solve the problem
    prob = Problem(Minimize(risk), [sum_entries(x) == 1, x >= 0, ret >= mu])
    prob.solve()
    
    # Get the values of interest
    minimal_risk = risk.value
    optimal_x = x.value
    
    # Handling rounding of x's
    optimal_x = np.around(optimal_x, decimals = 4)
    optimal_x =  np.asmatrix(optimal_x / sum(optimal_x))
    
    return(optimal_x)

In [38]:
def format_results(results, test, strategy, num_stocks):
    '''
    Format the results of the risk, returns into a data-frame 
    for plotting later
    
    Args: 
        results: numpy array with returns, risk results
        test: 1 or 0 depending on whether we are talking about in or 
                out of sample performance 
        strategy: string indicating which type of optimization approach was used
        num_stocks: the number of stocks selected for the portfolio
        
    Returns: 
        Dataframe with the following columns:
            return: the return for the portfolio
            risk: the risk for the portfolio
            strategy: see above for definition
            year: if training, the start year of the training period, 
                    if testing the evaluation year
            test: binary, 1 for out-of-sample, 0 for in-sample
            period: one of 6 values indicating which period of six years 
                        its in
            num_stocks: the number of stocks selected for the portfolio
    '''
    years = np.arange(1986, 2010 + 1)
    df = pd.DataFrame(results, columns = ['Return', 'Risk'])
    df['Sharpe'] = df['Return'] / df['Risk']
    df['strategy'] = strategy
    df['year'] = years + 6 if test == 1 else years
    df['test'] = test
    df['period'] = np.repeat(np.array([1, 2, 3, 4, 5]), 5)
    df['num_stocks'] = num_stocks
    return(df)

In [39]:
def risk_return(x, data):
    '''
    Calculate the risk, returns for a portfolio with a given set of weights
    
    Args:
        x: wieghts to be applied for the portfolio as a n-by-1 matrix
        data: an t-by-n pandas dataframe or numpy matrix.
        
    Returns:
        A tuple of two floats: one the risk and one the return
    '''
    
    r_hat = np.asmatrix(np.mean(data)).T
    Sigma = np.asmatrix(np.cov(data, rowvar=False))
    ret = (r_hat.T * x)[0,0]
    risk = (x.T * Sigma * x)[0,0]
    return(ret, risk)

In [40]:
# Set constants
START_YEAR = 1986
TAU = 1
NUM_PERIOD = 25

# Empty matrix for monthly returns (out of sample)
monthly_returns = np.asmatrix(np.zeros(shape = (12*NUM_PERIOD, 4)))

# Empty matrix for number of stocks selected
num_min_var = np.asmatrix(np.zeros(shape = (NUM_PERIOD, 1)))
num_factor = np.asmatrix(np.zeros(shape = (NUM_PERIOD, 1)))

In [41]:
# Get data
factors3 = read_factors3()
factors5 = read_factors5()
stocks = read_stocks()

# Number of stocks
n = stocks.shape[1]

# Calculate the equal weights vector
x_equal = get_equal_weights(n)

for i in range(NUM_PERIOD):
    
    # Set the period starting point
    start_year = START_YEAR + i
    
    # Get train, test split
    training, testing = train_test_split(stocks, start_year)
    train_factor3, test_factor3 = train_test_split(factors3, start_year)
    train_factor5, test_factor5 = train_test_split(factors5, start_year)
    
    # 1. Calculate the annual risk, returns for naive approach
    ret_equal, risk_equal = risk_return(x_equal, training)
        
    # 2. Estimate the covariance matrix matrix, returns vector from the training data
    r_hat, Sigma = ret_cov_est(training)
    
    # 3. Calulate the factor loadings, factor covariance matrix, and 
    # idiosyncratic risk from the training data
    F3, Sigma_tilde3, D3 = fama_and_french(training, train_factor3)
    F5, Sigma_tilde5, D5 = fama_and_french(training, train_factor5)
    
    # 4. Get the optimized weights
    # Note: I am using the naive portfolio for theshholds
    x_min_var = min_variance(r_hat = r_hat, Sigma = Sigma, tau = TAU, mu = ret_equal)
    x_factor3 = min_variance_factor(r_hat, F3, Sigma_tilde3, D3, mu = ret_equal)   
    x_factor5 = min_variance_factor(r_hat, F5, Sigma_tilde5, D5, mu = ret_equal)
    
    # 5. Calculate all 12 monthly returns for the out of sammple data
    # Note: Numpy selections are right exclusive (i.e. array[0:10] selects rows 0 through 9)
    monthly_returns[(12*i):(12*i + 12),0] = testing.values * x_equal
    monthly_returns[(12*i):(12*i + 12),1] = testing.values * x_min_var
    monthly_returns[(12*i):(12*i + 12),2] = testing.values * x_factor3 
    monthly_returns[(12*i):(12*i + 12),3] = testing.values * x_factor5
    
    # Store the number of stocks selected for record keeping purposes
    num_min_var[i,0] = sum(x_min_var > 0)
    num_factor[i,0] = sum(x_factor3 > 0)

In [42]:
# Format as data-frame
col_names = ['Equal', 'Mean-variance', 'Three-Factor', 'Five-Factor']
returns = pd.DataFrame(monthly_returns, columns = col_names)
returns['Date'] = pd.date_range('1/1/1991', periods=12*NUM_PERIOD, freq='M')

In [43]:
# Create periods
returns['Period'] = 1
returns.loc[returns['Date'] >= pd.datetime(2011, 1, 1), 'Period'] = 5
returns.loc[(returns['Date'] >= pd.datetime(2006, 1, 1)) & (returns['Date'] < pd.datetime(2011, 1, 1)), 'Period'] = 4
returns.loc[(returns['Date'] >= pd.datetime(2001, 1, 1)) & (returns['Date'] < pd.datetime(2006, 1, 1)), 'Period'] = 3
returns.loc[(returns['Date'] >= pd.datetime(1996, 1, 1)) & (returns['Date'] < pd.datetime(2001, 1, 1)), 'Period'] = 2
returns.loc[(returns['Date'] >= pd.datetime(1991, 1, 1)) & (returns['Date'] < pd.datetime(1996, 1, 1)), 'Period'] = 1

In [44]:
# Create date variables
returns.set_index('Date', inplace=True)
returns['year'] = returns.index.year
returns['month'] = returns.index.month
returns.head()

,Equal,Mean-variance,Three-Factor,Five-Factor,Period,year,month
Date,,,,,,,
1991-01-31,0.076230,-0.016445,0.053324,0.052417,1,1991,1
1991-02-28,0.101428,0.063873,0.080100,0.080025,1,1991,2
1991-03-31,0.043637,0.024132,0.033543,0.033102,1,1991,3
1991-04-30,0.018286,0.004339,0.010107,0.009918,1,1991,4
1991-05-31,0.040537,0.003743,0.036662,0.036018,1,1991,5


In [46]:
def annualized(x):
    return(np.exp(np.log(x + 1).sum()))

In [90]:
# Annualized returns
annualized_returns = returns.groupby(['Period', 'year'])[col_names].agg(annualized)
annualized_returns.head()

Equal  Mean-variance  Three-Factor  Five-Factor
Period year                                                    
1      1991  1.454159       1.212133      1.387198     1.381445
       1992  1.263247       1.105085      1.175827     1.175386
       1993  1.254158       1.242110      1.151043     1.151494
       1994  1.020566       0.975686      0.985586     0.985533
       1995  1.336725       1.228274      1.319557     1.319773

In [78]:
# Average annualized returns
avg_annualized = annualized_returns.groupby(level=0)[col_names].agg(np.mean)
avg_annualized

,Equal,Mean-variance,Three-Factor,Five-Factor
Period,,,,
1,1.265771,1.152658,1.203842,1.202726
2,1.191761,1.201017,1.183478,1.183515
3,1.194260,1.220118,1.155975,1.155696
4,1.139332,1.071700,1.102816,1.102854
5,1.122316,1.166599,1.126420,1.126945


In [79]:
# For the entire period: average annualized returns
total_annualized = annualized_returns.groupby([lambda idx: 'Total'])[col_names].agg(np.mean)
total_annualized

,Equal,Mean-variance,Three-Factor,Five-Factor
Total,1.182688,1.162418,1.154506,1.154347


In [80]:
def sharpe(x): 
    return(np.mean(x) / np.std(x))

In [81]:
def std(x):
    return(np.std(x, ddof = 1))

In [82]:
return_summary = returns.groupby(['Period'])[col_names].agg([np.mean, std, sharpe])
return_summary

Equal                     Mean-variance                      \
            mean       std    sharpe          mean       std    sharpe   
Period                                                                   
1       0.019642  0.027829  0.711758      0.011740  0.018734  0.631940   
2       0.015121  0.039315  0.387868      0.015265  0.031404  0.490205   
3       0.015015  0.041357  0.366130      0.016316  0.030980  0.531103   
4       0.010098  0.061710  0.165019      0.005420  0.036177  0.151096   
5       0.009719  0.038118  0.257126      0.013156  0.027029  0.490824   

       Three-Factor                     Five-Factor                      
               mean       std    sharpe        mean       std    sharpe  
Period                                                                   
1          0.015266  0.023908  0.643905    0.015196  0.023853  0.642465  
2          0.014348  0.037782  0.382956    0.014354  0.037917  0.381754  
3          0.012195  0.033204  0.370374    0.012174  0.033135  0.370505  
4          0.007766  0.053028  0.147685    0.007765  0.053046  0.147611  
5          0.010108  0.032716  0.311579    0.010153  0.032707  0.313054

In [83]:
return_total = returns.groupby([lambda idx: 'Total'])[col_names].agg([np.mean, np.std, sharpe])
return_total

Equal                     Mean-variance                      \
           mean       std    sharpe          mean       std    sharpe   
Total  0.013919  0.042977  0.324415      0.012379  0.029502  0.420317   

      Three-Factor                     Five-Factor                      
              mean       std    sharpe        mean       std    sharpe  
Total     0.011937  0.037225  0.321199    0.011928  0.037234  0.320898

In [ ]:
# Create date variables
returns.set_index('Date', inplace=True)
returns['year'] = returns.index.year
returns['month'] = returns.index.month
returns.head()